In [ ]:
import codecs
import os
import sys
import threading
import serial
from serial.tools.list_ports import comports

In [ ]:
import pynmea2

nmeaobj = pynmea2.parse(data.decode('utf-8'))
['%s: %s' % (nmeaobj.fields[i][0], nmeaobj.data[i]) 
     for i in range(len(nmeaobj.fields))]

In [ ]:
from a2gmeasurements import GpsSignaling

mySeptentrio = GpsSignaling(1)
mySeptentrio.serial_connect()
mySeptentrio.start_thread_serial()
mySeptentrio.sendCommandGps('sno, Stream 3, USB1, GGA, sec1')

In [ ]:
mySeptentrio.stop_thread_serial()
print(len(mySeptentrio.gps_rx_buffer), mySeptentrio.gps_rx_buffer[0])

In [ ]:
from a2gmeasurements import myAnritsuSpectrumAnalyzer

mySA = myAnritsuSpectrumAnalyzer()
mySA.spectrum_analyzer_connect()
rx_power =  mySA.retrieve_max_pow()
print(rx_power)
mySA.spectrum_analyzer_close()

In [1]:
from a2gmeasurements import GpsSignaling

mySeptentrioGPS = GpsSignaling(1)

mySeptentrioGPS.serial_connect()
mySeptentrioGPS.start_thread_serial()
#mySeptentrioGPS.start_gps_data_retrieval(msg_type='NMEA', nmea_type='HDT')
mySeptentrioGPS.start_gps_data_retrieval(msg_type='SBF')

setSBFOutput, Stream 1, USB1, PVTCartesian, sec1


In [2]:
mySeptentrioGPS.stop_gps_data_retrieval(msg_type='SBF')
mySeptentrioGPS.stop_thread_serial()

In [3]:
print(len(mySeptentrioGPS.SBF_frame_buffer), len(mySeptentrioGPS.NMEA_buffer))
if len(mySeptentrioGPS.SBF_frame_buffer) > 0:
    print(mySeptentrioGPS.SBF_frame_buffer[0])
if len(mySeptentrioGPS.NMEA_buffer) > 0:
    print(mySeptentrioGPS.NMEA_buffer[0])

5 0
{'SYNC2': b'@', 'CRC': 52592, 'ID': 20390, 'LEN': 96, 'TOW': 4294967295, 'WNc': 65535, 'MODE': 0, 'ERR': 1, 'X': -20000000000.0, 'Y': -20000000000.0, 'Z': -20000000000.0, 'Undulation': -20000000000.0, 'Vx': -20000000000.0, 'Vy': -20000000000.0, 'Vz': -20000000000.0, 'COG': -20000000000.0, 'RxClkBias': -20000000000.0, 'RxClkDrift': -20000000000.0, 'TimeSystem': 255, 'Datum': 255, 'NrSV': 255, 'WACorrInfo': 0, 'ReferenceID': 65535, 'MeanCorrAge': 65535, 'SignalInfo': 0, 'AlertFlag': 0, 'NrBases': 0, 'PPPInfo': 0, 'Latency': 18, 'HAccuracy': 65535, 'VAccuracy': 65535, 'Misc': 0, 'Padding': 0}
